# Weighted Gene Co-expression Network Analysis of LC M001 Liver Proteomics — WGCNA Module Detection

***Originally written by Tomasz Wilmanski, and modified by Kengo Watanabe***  

This Jupyter Notebook (with R kernel) performed the module detection part of the weighted gene co-expression network analysis (WGCNA; Langfelder, P. & Horvath, S. BMC Bioinform. 2008) on the preprocessed Longevity Consortium (LC) M001 proteomics dataset (adjusted with sex and age; analytes with 50% or less missingness), followed by the enrichment analysis on the hub proteins of WGCNA-identified modules using Gene Ontology Biological Process (GOBP) annotations.  

Input files:  
- Preprocessed analyte data: 230303_LC-M001-proteomics-WGCNA-ver2_Annotation_processed-data-for-WGCNA.tsv  
- Analyte metadata: 230303_LC-M001-proteomics-WGCNA-ver2_Annotation_analyte-metadata_UniProt.tsv  
- GOBP annotation metadata: 230303_LC-M001-proteomics-WGCNA-ver2_Annotation_module-metadata_QuickGO-GOBP-min-n4-cov50.tsv  

Output figures and tables:  
- Figure 3d  
- Supplementary Figure 2c  
- Supplementary Data 2, 3  

Original notebook (memo for my future tracing):  
- dalek:\[JupyterLab HOME\]/230303_LC-M001-proteomics-WGCNA-ver2/230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module.ipynb  

In [1]:
library("tidyverse")
options(repr.plot.width=5, repr.plot.height=5)#Default=7x7

#Bioconductor
for (package in c("WGCNA", "clusterProfiler", "enrichplot")) {
    #if (!requireNamespace("BiocManager", quietly=TRUE))
    #    install.packages("BiocManager")
    #BiocManager::install(package)
    eval(bquote(library(.(package))))
    print(str_c(package, ": ", as.character(packageVersion(package))))
}
options(stringsAsFactors=FALSE)#Required for WGCNA
#CRAN
for (package in c("readxl", "openxlsx")) {
    #install.packages(package)
    eval(bquote(library(.(package))))
    print(str_c(package, ": ", as.character(packageVersion(package))))
}

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




[1] "WGCNA: 1.71"


Registered S3 method overwritten by 'ggtree':
  method      from 
  identify.gg ggfun

clusterProfiler v4.2.2  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use clusterProfiler in published research, please cite:
T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal enrichment tool for interpreting omics data. The Innovation. 2021, 2(3):100141


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:purrr’:

    simplify


The following object is masked from ‘package:stats’:

    filter




[1] "clusterProfiler: 4.2.2"
[1] "enrichplot: 1.14.2"
[1] "readxl: 1.4.2"
[1] "openxlsx: 4.2.5.2"


## 1. Prepare dataset and metadata

In [ ]:
#Import analyte data
fileDir <- "./ExportData/"
ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_Annotation_"
fileName <- "processed-data-for-WGCNA.tsv"
data <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t") %>%
    column_to_rownames(var="UniProtID") %>%#For easily applying to the WGCNA functions
    t()

print(str_c("nrow: ", nrow(data)))
head(data)
data[, 1:5] %>% summary()

In [ ]:
#Import analyte metadata
fileDir <- "./ExportData/"
ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_Annotation_"
fileName <- "analyte-metadata_UniProt.tsv"
meta_analyte <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t") %>%
    dplyr::select(UniProtID, UniProtName, GeneSymbol)

print(str_c("nrow: ", nrow(meta_analyte)))
head(meta_analyte)
print(str_c("Unique analytes: ", length(unique(meta_analyte$UniProtID))))
print(str_c("Unique labels: ", length(unique(meta_analyte$GeneSymbol))))

In [ ]:
#Import annotation metadata
fileDir <- "./ExportData/"
ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_Annotation_"
fileName <- "module-metadata_QuickGO-GOBP-min-n4-cov50.tsv"
meta_annot <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t")

#Check
print(str_c("nrow: ", nrow(meta_annot)))
head(meta_annot)
print(str_c("Unique analytes: ", length(unique(meta_annot$UniProtID))))
print(str_c("Unique modules: ", length(unique(meta_annot$ModuleID))))

## 2. WGCNA

### 2-1. Missingness filter

> This preprocessing was already done.  

### 2-2. Network construction

#### 2-2-1. Choose the soft-thresholding power

> Choose a beta value to approximate a scale-free topology.  

In [ ]:
#Choose a set of soft-thresholding powers
powers <- 1:20
cutoff <- 0.8
#Call the network topology analysis function
sft <- pickSoftThreshold(data, powerVector=powers, verbose=5, RsquaredCut=cutoff,
                         corOptions=c(use="p", method="spearman"), networkType="signed hybrid")

#Plot the result
options(repr.plot.width=9, repr.plot.height=5)
par(mfrow=c(1,2))
cex1 <- 0.8
##Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     xlab="Soft Threshold (power)", ylab="Scale Free Topology Model Fit, signed R^2", type="n",
     main=paste("Scale independence"))
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     labels=powers, cex=cex1, col="black")
##Line corresponds to using an R^2 cut-off of h
abline(h=cutoff, col="red")
##Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
     xlab="Soft Threshold (power)", ylab="Mean Connectivity", type="n",
     main=paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1, col="black")

print(str_c("Soft-thresholding power: ", sft$powerEstimate))

#### 2-2-2. Co-expression similarity and adjacency

> Calculate correlation network adjacency.  

In [ ]:
#Generate the adjacency matrix using the chosen soft-thresholding power
softPower <- sft$powerEstimate
adjacency <- adjacency(data, power=softPower,
                       corOptions=list(use="p", method="spearman"), type="signed hybrid")

print(str_c("nrow: ", nrow(adjacency)))
head(adjacency)

### 2-3. Module detection

#### 2-3-1. Topological Overlap Matrix (TOM)

> To minimize effects of noise and spurious associations, the adjacency is transformed into the topological overlap measure, and the corresponding dissimilarity is calculated.  

In [ ]:
#Turn adjacency into topological overlap
TOM <- TOMsimilarity(adjacency, TOMType="signed")
dissTOM <- 1 - TOM

print(str_c("nrow: ", nrow(dissTOM)))
head(dissTOM)

#### 2-3-2. Hierarchical clustering using TOM dissimilarity

In [ ]:
#Call the hierarchical clustering function
geneTree <- hclust(as.dist(dissTOM), method="average")

#Plot the resulting clustering tree (dendrogram)
options(repr.plot.width=12, repr.plot.height=6)
plot(geneTree, xlab="", sub="", main="Gene clustering on TOM-based dissimilarity",
     labels=FALSE, hang=0.04)

> In the clustering tree (dendrogram), each leaf corresponds to a gene. Branches of the dendrogram group together densely interconnected, highly co-expressed genes. Module identification amounts to the identification of individual branches (”cutting the branches off the dendrogram”). There are several methods for branch cutting; the standard method of WGCNA package is Dynamic Tree Cut from the package dynamicTreeCut.  

In [ ]:
#We like large modules, so we set the minimum module size relatively high
minModuleSize <- 50
#Module identification using dynamic tree cut
dynamicMods <- cutreeDynamic(dendro=geneTree, distM=dissTOM,
                             deepSplit=4, pamStage=TRUE, pamRespectsDendro=FALSE,
                             minClusterSize=minModuleSize)
table(dynamicMods)

> The above list shows each module size. Label 0 is reserved for unassigned genes.  

In [ ]:
#Convert numeric lables into colors
dynamicColors <- labels2colors(dynamicMods)
table(dynamicColors)

#Plot the dendrogram and colors underneath
options(repr.plot.width=12, repr.plot.height=6)
plotDendroAndColors(geneTree, dynamicColors, "Dynamic Tree Cut",
                    dendroLabels=FALSE, hang=0.03,
                    addGuide=TRUE, guideHang=0.05,
                    main="Gene dendrogram and module colors")

#### 2-3-3. Merge similar modules based on eigengenes

> Dynamic Tree Cut may identify modules whose expression profiles are very similar. It would be prudent to merge such modules since their genes are highly co-expressed. To quantify co-expression similarity of entire modules, their eigengenes are calculated and clustered based on their correlation.  

In [ ]:
#Calculate eigengenes
MEList <- moduleEigengenes(data, colors=dynamicColors, impute=TRUE, nPC=2)
MEs <- MEList$eigengenes
print(str_c("nrow: ", nrow(MEs)))
head(MEs)

#Check the explained variance
temp_df <- MEList$varExplained
rownames(temp_df) <- c("PC1", "PC2")
colnames(temp_df) <- str_replace(names(MEList$eigengenes), "^ME", "")
t(temp_df)

#Calculate dissimilarity of module eigengenes
MEDiss <- 1 - cor(MEs, use="pairwise.complete.obs")

#Cluster module eigengenes
METree <- hclust(as.dist(MEDiss), method="average")

#Plot the result
options(repr.plot.width=10, repr.plot.height=5)
plot(METree, main="Clustering of module eigengenes",
     xlab="", sub="")
MEDissThres <- 0.1
abline(h=MEDissThres, col="red")

In [ ]:
#Call an automatic merging function
merge <- mergeCloseModules(data, dynamicColors, cutHeight=MEDissThres, verbose=3)

#Eigengenes of the new merged modules
mergedMEs <- merge$newMEs
print(str_c("nrow: ", nrow(mergedMEs)))
head(mergedMEs)

#Update the exparained variance
MEList <- moduleEigengenes(data, colors=merge$colors, impute=TRUE, nPC=2)
temp_df <- MEList$varExplained
rownames(temp_df) <- c("PC1", "PC2")
colnames(temp_df) <- str_replace(names(MEList$eigengenes), "^ME", "")
t(temp_df)

#Update the module eigengene clustering
##Calculate dissimilarity of module eigengenes
MEDiss <- 1 - cor(mergedMEs, use="pairwise.complete.obs")
##Cluster module eigengenes
METree <- hclust(as.dist(MEDiss), method="average")
##Plot the result
options(repr.plot.width=10, repr.plot.height=5)
plot(METree, main="Clustering of module eigengenes",
     xlab="", sub="")

In [ ]:
#The merged module colors
mergedColors <- merge$colors
table(mergedColors)

#Plot the dendrogram and module colors
options(repr.plot.width=12, repr.plot.height=6)
plotDendroAndColors(geneTree, cbind(dynamicColors, mergedColors),
                    c("Dynamic Tree Cut", "Merged dynamic"),
                    dendroLabels=FALSE, hang=0.03,
                    addGuide=TRUE, guideHang=0.05,
                    main="Gene dendrogram and module colors")

#### 2-3-4. Update/save

In [ ]:
#Rename
moduleColors <- mergedColors
MEs <- mergedMEs

#Construct numerical labels corresponding to the colors
colorOrder <- c("grey", standardColors(50))
moduleLabels <- match(moduleColors, colorOrder)-1

In [ ]:
#Prepare the module assignment table
module_tbl <- tibble(UniProtID=colnames(data),
                     ModuleID=str_to_title(moduleColors),
                     ModuleNumber=moduleLabels) %>%
    dplyr::left_join(meta_analyte, ., by="UniProtID")
print(str_c("nrow: ", nrow(module_tbl)))
head(module_tbl)

#Save module assignments
#-> After adding intramodular connectivity to this table later

In [ ]:
#Clean the module eigengene table
temp_df <- MEs %>%
    rownames_to_column(var="SampleID")
names(temp_df)[2:ncol(temp_df)] <- names(temp_df)[2:ncol(temp_df)] %>%
    str_replace(., "^ME", "") %>%
    str_to_title(.)
print(str_c("nrow: ", nrow(temp_df)))
head(temp_df)
    
#Save module eigengene values
fileDir <- "./ExportData/"
ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
fileName <- "module-eigengenes.tsv"
temp_df %>% write_tsv(str_c(fileDir,ipynbName,fileName))

### 2-4. Relathionship between each analyte and module

#### 2-4-1. Module membership

> For each module and each gene, a quantitative measure of module membership (MM) is defined as the correlation between the module eigengene and the gene expression profile. This allows us to quantify the similarity of all input genes (irrespective of their original module membership) to every module.  

In [ ]:
#Calculate the numbers of genes and samples
nGenes <- ncol(data)
nSamples <- nrow(data)

#Names (colors) of the modules
modNames = substring(names(MEs), 3)

#Calculate module membership
geneModuleMembership <- as.data.frame(cor(data, MEs, use="p"))
#names(geneModuleMembership) <- paste("MM", modNames, sep="")
names(geneModuleMembership) <- str_to_title(modNames)
print(str_c("nrow: ", nrow(geneModuleMembership)))
head(geneModuleMembership)

#Calculate statisitcal significance of module membership
MMPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples))
#names(MMPvalue) <- paste("p.MM", modNames, sep="")
names(MMPvalue) <- str_to_title(modNames)
print(str_c("nrow: ", nrow(MMPvalue)))
head(MMPvalue)

In [ ]:
#Save module membership summary
fileDir <- "./ExportData/"
ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
fileName <- "module-membership.tsv"
geneModuleMembership %>% rownames_to_column(var="UniProtID") %>% write_tsv(str_c(fileDir,ipynbName,fileName))
fileName <- "module-membership_pvalue.tsv"
MMPvalue %>% rownames_to_column(var="UniProtID") %>% write_tsv(str_c(fileDir,ipynbName,fileName))

#### 2-4-2. Intramodular connectivity

> This is not included in the official tutorial section, but prepared based on the last "Export of networks to external software" section and Tom's notebook.  

In [ ]:
#Prepare target modules
targets <- modNames[modNames!="grey"]

#Repeat for each module
temp_tbl <- tibble()
for (module in targets) {
    #Select module probes
    probes <- colnames(data)
    inModule <- (moduleColors==module)
    modProbes <- probes[inModule]
    
    #Select the corresponding Topological Overlap
    modTOM <- TOM[inModule, inModule]
    dimnames(modTOM) <- list(modProbes, modProbes)
    
    #Calculate intramodular connectivity
    IMConn <- softConnectivity(data[, modProbes], power=softPower,
                               corOptions=list(use="p", method="spearman"), type="signed hybrid")
    
    #Summary table
    connectivity <- tibble(ModuleID=str_to_title(module),
                           UniProtID=modProbes,
                           IntramodularConnectivity=IMConn,
                           TOMsimilaritySum=rowSums(modTOM))
    print(str_c(module," module: ", nrow(connectivity)))
    print(head(connectivity))#Due to within for-loop
    
    #Add to the overall table
    temp_tbl <- dplyr::bind_rows(temp_tbl, connectivity)
}

print(str_c("Total nrow: ", nrow(temp_tbl)))
head(temp_tbl)

In [ ]:
#Update the module assignment table
module_tbl <- dplyr::left_join(module_tbl, temp_tbl, by=c("UniProtID", "ModuleID"))
print(str_c("nrow: ", nrow(module_tbl)))
head(module_tbl)
module_tbl %>% dplyr::filter(is.na(IntramodularConnectivity))

#Save module assignments
fileDir <- "./ExportData/"
ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
fileName <- "module-assignments.tsv"
module_tbl %>% write_tsv(str_c(fileDir,ipynbName,fileName))

In [ ]:
#Check top15 analytes
for (module in targets) {
    print(module)
    module_tbl %>%
        select(-ModuleNumber) %>%
        dplyr::filter(ModuleID==str_to_title(module)) %>%
        dplyr::arrange(desc(IntramodularConnectivity)) %>%
        head(., n=15) %>%
        print()#Due to within for-loop
}

### 2-5. Export module networks

> Instead of VisANT, Cytoscape version is used here. Also, intra-module connectivity is calculated within this code.  

In [ ]:
#Repeat for each module
for (module in targets) {
    #Select module probes
    probes <- colnames(data)
    inModule <- (moduleColors==module)
    modProbes <- probes[inModule]
    print(str_c(module," module: ", length(modProbes)))
    modGenes <- meta_analyte$GeneSymbol[match(modProbes, meta_analyte$UniProtID)]
    
    #Select the corresponding Topological Overlap
    modTOM <- TOM[inModule, inModule]
    dimnames(modTOM) <- list(modProbes, modProbes)
    
    #Export the network into edge and node list files Cytoscape can read
    fileDir <- "./ExportData/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("CytoscapeInput-",module)
    cyt <- exportNetworkToCytoscape(modTOM,
                                    edgeFile=str_c(fileDir,ipynbName,fileName,"-edges.txt"),
                                    nodeFile=str_c(fileDir,ipynbName,fileName,"-nodes.txt"),
                                    weighted=TRUE,
                                    threshold=0,
                                    nodeNames=modProbes,
                                    altNodeNames=modGenes,
                                    nodeAttr=str_to_title(moduleColors[inModule]))
}

## 3. Enrichment analysis for module members

### 3-1. Save result objects

In [ ]:
#Create a workbook object to save as one single .xlsx file
workbook <- createWorkbook()

#Summarize results per WGCNA module
nMembers_vec <- c()
nMappedMembers_vec <- c()
nMemberModules_vec <- c()
res_list <- list()
for (i in 1:length(targets)) {
    #Prepare input for enricher()
    module <- str_to_title(targets[i])#WGCNA module
    members <- module_tbl %>%
        dplyr::filter(ModuleID==module) %>%
        .$GeneSymbol#Able to use instead of UniProtID because duplicate issues were resolved
    backgrounds <- meta_annot %>%
        dplyr::select(ModuleID, GeneSymbol)#Here, not WGCNA module but a priori module (functional annotation)
    labels <- meta_annot %>%
        dplyr::select(ModuleID, ModuleName) %>%#Here, not WGCNA module but a priori module (functional annotation)
        dplyr::distinct()
    print(str_c(module,": ",length(members)," members"))
    
    #Save info
    nMembers_vec <- c(nMembers_vec, length(members))
    temp <- backgrounds %>% dplyr::filter(GeneSymbol %in% members)
    nMappedMembers_vec <- c(nMappedMembers_vec, length(unique(temp$GeneSymbol)))
    nMemberModules_vec <- c(nMemberModules_vec, length(unique(temp$ModuleID)))
    
    #Enrichment analysis
    temp <- enricher(gene=members,
                     pvalueCutoff=1.0,#To export all
                     pAdjustMethod="BH",
                     #universe=backgrounds,#Already managed
                     minGSSize=4,
                     maxGSSize=10000,
                     qvalueCutoff=1.0,#To export all
                     TERM2GENE=backgrounds,
                     TERM2NAME=labels)
    
    #Add the summary table to the workbook object as an independent sheet
    if (is.data.frame(temp[])) {
        temp1 <- tibble(temp[]) %>%
            dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                          Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
    } else {
        temp1 <- tibble(`n/a`=NA)
    }
    addWorksheet(workbook, sheetName=module)
    writeData(workbook, module, temp1)
    
    #Add result object to list
    res_list <- c(res_list, list(temp))
}

#Save the workbook as one single .xlsx file
fileDir <- "./ExportData/"
ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
fileName <- "clusterProfiler-members.xlsx"
saveWorkbook(workbook, file=str_c(fileDir,ipynbName,fileName), overwrite=TRUE)

print(str_c("nObjects: ", as.character(length(res_list))))

### 3-2. Check results

#### 3-2-1. Greenyellow

In [ ]:
obj_i <- 1

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-2. Turquoise

In [ ]:
obj_i <- 2

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-3. Yellow

In [ ]:
obj_i <- 3

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-4. Black

In [ ]:
obj_i <- 4

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-5. Blue

In [ ]:
obj_i <- 5

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-6. Purple

In [ ]:
obj_i <- 6

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-7. Salmon

In [ ]:
obj_i <- 7

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-8. Tan

In [ ]:
obj_i <- 8

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-9. Brown

In [ ]:
obj_i <- 9

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-2-10. Pink

In [ ]:
obj_i <- 10

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nMembers <- nMembers_vec[obj_i]
nMappedMembers <- nMappedMembers_vec[obj_i]
nMemberModules <- nMemberModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of members: ",as.character(nMembers)))
print(str_c(" - # of members mapped to any annotation: ",as.character(nMappedMembers)))
print(str_c(" - # of annotations having any member: ",as.character(nMemberModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Members=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedMember=geneID, nMappedMembers=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 25), breaks=seq(0, 25, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of members",
             y="", title=str_c("Enriched GOBP in ",module," module members")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-all.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

## 4. Enrichment analysis for top hub proteins

### 4-1. Save result objects

In [ ]:
topQ <- 0.9#Focus on the top hubs

#Create a workbook object to save as one single .xlsx file
workbook <- createWorkbook()

#Summarize results per WGCNA module
nHubs_vec <- c()
nMappedHubs_vec <- c()
nHubModules_vec <- c()
res_list <- list()
for (i in 1:length(targets)) {
    #Prepare input for enricher()
    module <- str_to_title(targets[i])#WGCNA module
    hubs <- module_tbl %>%
        dplyr::filter(ModuleID==module) %>%
        dplyr::filter(IntramodularConnectivity>=quantile(IntramodularConnectivity, topQ)) %>%
        .$GeneSymbol#Able to use instead of UniProtID because duplicate issues were resolved
    backgrounds <- meta_annot %>%
        dplyr::select(ModuleID, GeneSymbol)#Here, not WGCNA module but a priori module (functional annotation)
    labels <- meta_annot %>%
        dplyr::select(ModuleID, ModuleName) %>%#Here, not WGCNA module but a priori module (functional annotation)
        dplyr::distinct()
    print(str_c(module,": ",length(hubs)," hubs"))
    
    #Save info
    nHubs_vec <- c(nHubs_vec, length(hubs))
    temp <- backgrounds %>% dplyr::filter(GeneSymbol %in% hubs)
    nMappedHubs_vec <- c(nMappedHubs_vec, length(unique(temp$GeneSymbol)))
    nHubModules_vec <- c(nHubModules_vec, length(unique(temp$ModuleID)))
    
    #Enrichment analysis
    temp <- enricher(gene=hubs,
                     pvalueCutoff=1.0,#To export all
                     pAdjustMethod="BH",
                     #universe=backgrounds,#Already managed
                     minGSSize=4,
                     maxGSSize=10000,
                     qvalueCutoff=1.0,#To export all
                     TERM2GENE=backgrounds,
                     TERM2NAME=labels)
    
    #Add the summary table to the workbook object as an independent sheet
    if (is.data.frame(temp[])) {
        temp1 <- tibble(temp[]) %>%
            dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                          Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
    } else {
        temp1 <- tibble(`n/a`=NA)
    }
    addWorksheet(workbook, sheetName=module)
    writeData(workbook, module, temp1)
    
    #Add result object to list
    res_list <- c(res_list, list(temp))
}

#Save the workbook as one single .xlsx file
fileDir <- "./ExportData/"
ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
fileName <- "clusterProfiler-hubs.xlsx"
saveWorkbook(workbook, file=str_c(fileDir,ipynbName,fileName), overwrite=TRUE)

print(str_c("nObjects: ", as.character(length(res_list))))

### 4-2. Check results

#### 4-2-1. Greenyellow

In [ ]:
obj_i <- 1

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-2. Turquoise

In [ ]:
obj_i <- 2

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-3. Yellow

In [ ]:
obj_i <- 3

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-4. Black

In [ ]:
obj_i <- 4

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-5. Blue

In [ ]:
obj_i <- 5

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-6. Purple

In [ ]:
obj_i <- 6

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-7. Salmon

In [ ]:
obj_i <- 7

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-8. Tan

In [ ]:
obj_i <- 8

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-9. Brown

In [ ]:
obj_i <- 9

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 4-2-10. Pink

In [ ]:
obj_i <- 10

#Retreive results
module <- str_to_title(targets[obj_i])#WGCNA module
nHubs <- nHubs_vec[obj_i]
nMappedHubs <- nMappedHubs_vec[obj_i]
nHubModules <- nHubModules_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(module)
print(str_c(" - # of hubs: ",as.character(nHubs)))
print(str_c(" - # of hubs mapped to any annotation: ",as.character(nMappedHubs)))
print(str_c(" - # of annotations having any hub: ",as.character(nHubModules)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::rename(AnnotationID=ID, AnnotationName=Description, Ratio2Hubs=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedHub=geneID, nMappedHubs=Count)
}

#Visualization
if (is.data.frame(res[])) {
    display <- tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%
        nrow()
} else {
    display <- 0
}
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=0.5, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 15), breaks=seq(0, 15, by=5), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=60)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), oob=scales::squish,
                             name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of hubs",
             y="", title=str_c("Enriched GOBP in ",module," module hubs")) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica", lineheight=0.75),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=9, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "230303_LC-M001-proteomics-WGCNA-ver2_WGCNA-module_"
    fileName <- str_c("enrichment-",module,"-hub.pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=9, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

# — Session information —

In [45]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/envs/arivale-r/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] openxlsx_4.2.5.2      readxl_1.4.2          enrichplot_1.14.2    
 [4] clusterProfiler_4.2.2 WGCNA_1.71            fastcluster_1.2.3    
 [7] dynamicTreeCut_1.63-1 forcats_0.5.1         stringr_1.4.0        
[10] dplyr_1.0.9           purrr_0.3.4           readr_2.1.2          
[13] tidyr_1.2.0           tib